<a href="https://colab.research.google.com/github/KIQVinicius/Data-Science/blob/master/AnomaliasJob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
import pandas as pd
from datetime import timedelta

In [133]:
import numpy as np
from datetime import date

In [134]:
dataset = pd.read_csv("MIPS1.csv")


In [135]:
porcentagem = float(dataset['Porcentagem'][1])/100 + 1
dataset.columns = ['Index', 'JOBNAME', 'DATE', 'Valor_MIPS', 'Porcentagem', 'Chave']

In [136]:
dataset = dataset.drop(columns=["Porcentagem", "Chave"], axis=1)

In [137]:
dataset['DATE'] = pd.to_datetime(dataset['DATE'])
dataset = dataset.sort_values(['DATE'])
dataset = dataset.head(100000)

In [138]:
first_day = dataset['DATE'].min()
last_day = dataset['DATE'].max()

In [139]:
first_day

Timestamp('2019-01-11 00:00:00')

In [140]:
last_day

Timestamp('2019-09-10 00:00:00')

In [141]:
dt = last_day.date() - first_day.date()
dt.days

242

In [142]:
weeks = int(dt.days/7)
weeks

34

In [143]:
months = int(dt.days/30)
months

8

In [144]:
data = first_day
dataset['JOBNAME'].value_counts()

FZ78OK      23
FZ90DF03    23
FG95MR02    23
FG95MP06    23
FZ8618A     23
            ..
FP555C32     1
FP555C24     1
FJ0Q150E     1
FZ90DV92     1
FG92FI       1
Name: JOBNAME, Length: 9888, dtype: int64

In [125]:
frequencia = pd.DataFrame(dataset['JOBNAME'].value_counts())
frequencia.reset_index(inplace=True)
frequencia.columns = ['JOBNAME', 'Frequencia']

In [126]:
frequencia = frequencia.query("Frequencia > 1")

In [127]:
dataset = dataset[dataset['JOBNAME'].isin(frequencia['JOBNAME'])]
dataset.head()

,Index,JOBNAME,DATE,Valor_MIPS
192478,192478,BD26B8L,2019-01-11,1.14
196089,196089,FS0245SI,2019-01-11,2.88
196088,196088,FS0245SF,2019-01-11,0.01
196087,196087,FS0245NL,2019-01-11,3.64
196086,196086,FS0245NG,2019-01-11,0.18


In [128]:
dataset = dataset.drop(columns=["Index"], axis=1)
dataset['JOBNAME'].value_counts()

FQ20A90     23
FG95MP02    23
JESXCF      23
FG95MQBF    23
GSSA        23
            ..
FK20A58      2
FZ651PEH     2
IMSRDR       2
FA3150       2
FK61B8E      2
Name: JOBNAME, Length: 8647, dtype: int64

# Anomalia


In [129]:
def DetectarAnomalia(dataset, intervalo):
  if intervalo == "Mensal":
    periodo = months
    intervalo_de_dias = 30
  else:
    periodo = weeks
    intervalo_de_dias = 7
  data = first_day
  tabela_anomalia = dataset.iloc[0:0]
  dataset_semana_anterior = dataset.query("DATE >= " + " '"+ str(data)+ "' " + " and DATE < " + " '"+str(data+ timedelta(days = intervalo_de_dias)) + "' ")
  data = data+ timedelta(days = intervalo_de_dias)
  dataset_semana_anterior = dataset_semana_anterior[dataset_semana_anterior['Valor_MIPS']>0].groupby('JOBNAME')
  dataset_semana_anterior = pd.DataFrame(dataset_semana_anterior['Valor_MIPS'].mean())
  dataset_semana_anterior.reset_index(inplace =True)
  for i in range(0, periodo, 1):
    dataset_semana = dataset.query("DATE >= " + " '"+ str(data)+ "' " + " and DATE < " + " '"+str(data+ timedelta(days = intervalo_de_dias)) + "' ")
    #dataset = dataset.query("DATE >= " + " '"+ str(data+ timedelta(days = 30))+ "' ")
    print("-------------------------------------------------")
    #dataset_semana = dataset_semana[dataset_semana['Valor_MIPS']>0].groupby('JOBNAME')
    #dataset_semana = pd.DataFrame(dataset_semana['Valor_MIPS'].mean())
    dataset_semana_anterior = dataset_semana_anterior[dataset_semana_anterior['JOBNAME'].isin(dataset_semana['JOBNAME'])]
    for index, row in dataset_semana_anterior.iterrows():
      tabela_anomalia = tabela_anomalia.append(dataset_semana.query("JOBNAME == "+"'"+row['JOBNAME']+"'" + " and Valor_MIPS > "+ str(float(row['Valor_MIPS']) * porcentagem)))
      tabela_anomalia = tabela_anomalia.append(dataset_semana.query("JOBNAME == "+"'"+row['JOBNAME']+"'" + " and Valor_MIPS < "+ str(float(row['Valor_MIPS']) * porcentagem)))
    print("Anomalias até " + str(data + timedelta(days = intervalo_de_dias) )[0:10]+": "  + str(tabela_anomalia.shape[0]))
    dataset_semana_anterior = dataset_semana[dataset_semana['Valor_MIPS']>0].groupby('JOBNAME')
    dataset_semana_anterior = pd.DataFrame(dataset_semana_anterior['Valor_MIPS'].mean())
    dataset_semana_anterior.reset_index(inplace=True)
    data = data+ timedelta(days = intervalo_de_dias)
  tabela_anomalia['Anomalia'] = intervalo
  return tabela_anomalia


In [145]:
tabela_mensal = DetectarAnomalia(dataset, "Mensal")

-------------------------------------------------
Anomalias até 2019-03-12: 13743
-------------------------------------------------


KeyboardInterrupt: ignored

In [131]:
tabela_semanal = DetectarAnomalia(dataset, "Semanal")

-------------------------------------------------
Anomalias até 2019-01-25: 0
-------------------------------------------------
Anomalias até 2019-02-01: 0
-------------------------------------------------
Anomalias até 2019-02-08: 0
-------------------------------------------------
Anomalias até 2019-02-15: 0
-------------------------------------------------
Anomalias até 2019-02-22: 0
-------------------------------------------------
Anomalias até 2019-03-01: 0
-------------------------------------------------
Anomalias até 2019-03-08: 0
-------------------------------------------------
Anomalias até 2019-03-15: 0
-------------------------------------------------
Anomalias até 2019-03-22: 0
-------------------------------------------------
Anomalias até 2019-03-29: 0
-------------------------------------------------
Anomalias até 2019-04-05: 0
-------------------------------------------------
Anomalias até 2019-04-12: 0
-------------------------------------------------


KeyboardInterrupt: ignored

In [23]:
tabela_final = tabela_semanal.iloc[0:0]
tabela_final = tabela_final.append(tabela_mensal)
print(tabela_final.shape)
tabela_final = tabela_final.append(tabela_semanal)
print(tabela_final.shape)

(0, 4)
(0, 4)


In [152]:
a = list(np.arange(1, months, months/4))
a

[1.0, 3.0, 5.0, 7.0]